In [120]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

Carregando o dataset sobre os exoplanetas

In [89]:
table_PS = pd.read_csv("data/PS_2024.10.05_09.10.06.csv", skiprows=96)

Criando um novo dataframe com base na coluna default_flag
Deixando apenas os exoplanetas confiaveis para futura analise

In [104]:
confiavel_PS = table_PS[table_PS["default_flag"]==1]

Removendo colunas que comparam o tamanho do exoplanetas com jupiter

In [103]:
confiavel_PS = confiavel_PS.drop("pl_radj", axis=1)
confiavel_PS = confiavel_PS.drop("pl_radjerr1", axis=1)
confiavel_PS = confiavel_PS.drop("pl_radjerr2", axis=1)
confiavel_PS = confiavel_PS.drop("pl_radjlim", axis=1)
confiavel_PS = confiavel_PS.drop("pl_bmassj", axis=1)
confiavel_PS = confiavel_PS.drop("pl_bmassjerr1", axis=1)
confiavel_PS = confiavel_PS.drop("pl_bmassjerr2", axis=1)
confiavel_PS = confiavel_PS.drop("pl_bmassjlim", axis=1)

# Está na zona habitavel?

Fazendo uma busca em planetas com base na insolacao deles

Considerando um valor entre 0.7 e 1.5, que pode indicar que o exoplaneta esta na zona habitavel

In [92]:
habitaveis = confiavel_PS.query("0.7 <= pl_insol <= 1.5").sort_values(by="pl_insol")
habitaveis[["pl_name", "pl_insol"]]

,pl_name,pl_insol
1902,HD 109286 b,0.71
14740,Kepler-1649 c,0.75
3030,HD 27969 b,0.76
14784,Kepler-1652 b,0.81
34673,TOI-700 d,0.85
25454,Kepler-443 b,0.86
14860,Kepler-1661 b,0.88
32934,LP 890-9 c,0.91
14795,Kepler-1653 b,0.98
2646,HD 191939 g,0.99


# Órbita do exoplaneta

Ordena os valores com base na órbita, onde:
- Proximo a 0 a órbita é mais circular
- Próximo a 1 a órbita é mais hiperbólica
- Entre os dois a órbita é elíptica

A visualização ignora linhas com valores nulos

In [93]:
orbita = confiavel_PS[confiavel_PS["pl_orbeccen"].notna()].sort_values(by="pl_orbeccen")
orbita[["pl_name","pl_orbeccen"]]

,pl_name,pl_orbeccen
33851,TOI-1347 c,0.0000
6570,Kepler-101 c,0.0000
35507,WASP-24 b,0.0000
1986,HD 116029 b,0.0000
6380,Kepler-10 b,0.0000
...,...,...
3560,HD 80606 b,0.9300
33652,TIC 241249530 b,0.9412
3840,HIP 66074 b,0.9480
3046,HD 28254 b,0.9500


# Distancia da terra até

Criando uma nova coluna no dataset que mostra a distancia da terra para o exoplaneta em km

No caso, transforma de Parsec para km

In [94]:
PARSEC = (30857 * 10e12)
confiavel_PS["sy_dist_km"] = confiavel_PS["sy_dist"] * PARSEC

Seleciona os planetas com temperatura entre 273 e 373 Kelvin
e os ordena da menor para maior temperatura

A temperatura escolhida é aquela onde pode existir água no seu estado líquido

A visualização ignora linhas com valores nulos

In [95]:
agua_liquida = confiavel_PS[confiavel_PS["pl_eqt"].notna()].query("273 <= pl_eqt <= 373").sort_values("pl_eqt")
agua_liquida[["pl_name", "pl_eqt"]]

,pl_name,pl_eqt
27890,Kepler-61 b,273.0
36321,eps Ind A b,275.0
18433,Kepler-1996 c,276.0
34820,Teegarden's Star b,277.0
2646,HD 191939 g,278.0
...,...,...
2138,HD 137496 c,370.0
226,CFBDSIR J145829+101343 b,370.0
16433,Kepler-1801 c,371.0
4945,K2-3 c,372.0


# Composição Atmosférica e Condições de Superfície

Colunas relacionadas: pl_eqt, pl_insol, pl_bmasse, st_teff, st_rad
A combinação das colunas relacionadas à massa e raio do planeta com a temperatura da estrela (coluna st_teff) e seu tamanho (coluna st_rad) pode ajudar a estimar a composição atmosférica ou condições de superfície.
Pergunta: Quais são as possíveis condições atmosféricas do planeta? Ele possui uma atmosfera densa ou rarefeita?

pl_eqt - A temperatura de equilíbrio do planeta modelada como um corpo negro aquecido apenas por sua estrela        hospedeira, ou, no caso de planetas com imagem direta, a temperatura efetiva do planeta necessária para corresponder à luminosidade medida, caso o planeta fosse um corpo negro

pl_bmasse - Melhor estimativa de massa do planeta disponível, em ordem de preferência: Massa, Msin(i)/sin(i), ou Msin(i), dependendo da disponibilidade, e medida em massas da Terra

st_teff - Temperatura da estrela modelada como um corpo negro emitindo a mesma quantidade total de radiação eletromagnética

st_rad - Comprimento de um segmento de linha do centro da estrela até sua superfície, medido em unidades do raio do Sol

In [137]:
confiavel_PS_filtered = confiavel_PS[confiavel_PS["pl_bmasse"].notna() & confiavel_PS["pl_rade"].notna()]

In [140]:
confiavel_PS_filtered["density"] = confiavel_PS_filtered["pl_bmasse"] / (((4/3) * np.pi) * (confiavel_PS_filtered["pl_rade"] ** 3))
confiavel_PS_filtered
c_atm = confiavel_PS_filtered[["pl_name", "pl_eqt", "pl_bmasse", "st_teff", "st_rad", "pl_bmassprov", "density"]]
c_atm.sort_values(by="density").head(10)


/var/folders/5_/w_2wxvn17670gd91zgz7yfqw0000gp/T/ipykernel_22200/1776854283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  confiavel_PS_filtered["density"] = confiavel_PS_filtered["pl_bmasse"] / (((4/3) * np.pi) * (confiavel_PS_filtered["pl_rade"] ** 3))


,pl_name,pl_eqt,pl_bmasse,st_teff,st_rad,pl_bmassprov,density
26361,Kepler-51 c,439.0,4.00000,6018.0,0.94,Mass,0.001310
1759,HD 100546 b,NaN,2701.54146,10500.0,NaN,Mass,0.001394
26346,Kepler-51 b,543.0,2.10000,6018.0,0.94,Mass,0.001401
26386,Kepler-51 d,381.0,7.60000,6018.0,0.94,Mass,0.001988
1387,HAT-P-67 b,1903.0,108.06220,6406.0,2.55,Mass,0.002021
35454,WASP-193 b,1254.0,44.17815,6078.0,1.24,Mass,0.002387
33874,TOI-1420 b,957.0,25.10000,5510.0,0.92,Mass,0.003565
5683,KELT-11 b,NaN,54.34893,5375.0,2.69,Mass,0.003745
30185,Kepler-79 d,NaN,6.00000,6174.0,1.30,Mass,0.003902
35124,WASP-127 b,NaN,52.34660,NaN,1.33,Mass,0.003938


### 1. **Temperatura da estrela modelada como um corpo negro emitindo a mesma quantidade total de radiação eletromagnética**
   - **Explicação:** Esta expressão refere-se à temperatura efetiva de uma estrela, que é uma forma de descrever sua temperatura superficial. Um corpo negro é um objeto teórico que absorve toda a radiação que incide sobre ele e emite radiação térmica de acordo com sua temperatura. No contexto das estrelas, a temperatura é calculada a partir da quantidade total de radiação eletromagnética que a estrela emite. Isso é importante para entender a energia que a estrela produz e como essa energia se relaciona com a sua cor, luminosidade e evolução.

### 2. **Comprimento de um segmento de linha do centro da estrela até sua superfície, medido em unidades do raio do Sol**
   - **Explicação:** Este trecho refere-se ao raio de uma estrela, que é a distância do centro até a superfície. A unidade de medida é o raio do Sol (R☉), que serve como um padrão para comparação. A análise do raio é fundamental para determinar características físicas da estrela, como sua densidade e a força da gravidade em sua superfície. Além disso, o raio pode ajudar a classificar as estrelas em diferentes categorias (por exemplo, estrelas anãs, gigantes, supergigantes) e influenciar a habitabilidade de exoplanetas.

### 3. **Quantidade de matéria contida na estrela, medida em unidades de massas do Sol**
   - **Explicação:** Esta expressão diz respeito à massa estelar, que é a totalidade da matéria presente em uma estrela. A massa é medida em unidades da massa do Sol (M☉), que é um padrão de referência na astronomia. A massa de uma estrela tem implicações significativas em sua evolução, como a fase da vida em que se encontra, sua luminosidade e o tipo de final que terá (supernova, anã branca, etc.). Também influencia a gravidade que a estrela exerce sobre objetos ao seu redor, como planetas e outros corpos celestes.

### 4. **Aceleração gravitacional experimentada na superfície estelar**
   - **Explicação:** Esta frase se refere à gravidade superficial de uma estrela, que é a aceleração gravitacional que um objeto sentiria na superfície da estrela. Essa aceleração é influenciada pela massa da estrela e pelo seu raio. Em geral, uma estrela mais massiva e com um raio menor terá uma gravidade superficial mais alta. Compreender a gravidade superficial é essencial para estudar a estrutura interna da estrela, a dinâmica das camadas de gás em sua atmosfera, e como essa gravidade afeta a formação de planetas e outros corpos em seu entorno.

# Sistema Estelar (Número de Estrelas e Planetas)

Colunas: sy_snum, sy_pnum
Essas colunas indicam quantas estrelas e planetas estão no sistema do exoplaneta.
Pergunta: O planeta orbita uma estrela única ou faz parte de um sistema binário? Sistemas estelares múltiplos podem ter dinâmicas mais complexas que afetam a habitabilidade.

sy_snum - Número de estrelas no sistema planetário

sy_pnum - Número de planetas confirmados no sistema planetário

In [41]:
confiavel_PS[["pl_name", "sy_snum", "sy_pnum"]]

,pl_name,sy_snum,sy_pnum
2,11 Com b,2,1
3,11 UMi b,1,1
6,14 And b,1,1
11,14 Her b,1,2
22,16 Cyg B b,3,1
...,...,...,...
36404,ups And b,2,3
36411,ups And c,2,3
36416,ups And d,2,3
36419,ups Leo b,1,1


# Parâmetros Estelares (Propriedades da Estrela Hospedeira)


Colunas: st_teff, st_rad, st_mass, st_met, st_logg
Essas colunas descrevem as características físicas da estrela hospedeira, como temperatura, raio, massa, metalicidade (abundância de elementos mais pesados que hélio), e gravidade superficial.
Pergunta: A estrela é semelhante ao Sol ou muito diferente? A habitabilidade do planeta também depende muito da natureza da estrela hospedeira.

st_teff - Temperatura da estrela modelada como um corpo negro emitindo a mesma quantidade total de radiação eletromagnética

st_rad - Comprimento de um segmento de linha do centro da estrela até sua superfície, medido em unidades do raio do Sol

st_mass - Quantidade de matéria contida na estrela, medida em unidades de massas do Sol

st_logg - Aceleração gravitacional experimentada na superfície estelar

In [43]:
# interessante apenas tamanho e temperatura
# considere que o tamanho informa o raio da estrela
estrela = confiavel_PS[["pl_name", "st_teff", "st_rad", "st_mass", "st_logg"]]
estrela

,pl_name,st_teff,st_rad,st_mass,st_logg
2,11 Com b,4874.0,13.76,2.09,2.45
3,11 UMi b,4213.0,29.79,2.78,1.93
6,14 And b,4888.0,11.55,1.78,2.55
11,14 Her b,NaN,NaN,0.91,NaN
22,16 Cyg B b,5750.0,1.13,1.08,4.36
...,...,...,...,...,...
36404,ups And b,NaN,1.56,1.30,NaN
36411,ups And c,NaN,1.56,1.30,NaN
36416,ups And d,NaN,1.56,1.30,NaN
36419,ups Leo b,4836.0,11.22,1.48,2.46
